In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rating = pd.read_csv('BooksRating.csv',low_memory = False, on_bad_lines='skip')
overall = pd.read_csv('BooksDataset.csv',low_memory = False, on_bad_lines='skip')

In [ ]:
rating.head()

In [ ]:
overall.head()

In [ ]:
overall.shape

In [ ]:
rating.shape

In [ ]:
book = overall.merge(rating, left_on='Title', right_on='title')

In [ ]:
book.head()

In [ ]:
book.shape

In [ ]:
book.info()

In [ ]:
book.describe()

In [ ]:
book = book[['Title','authors','Description','Category','Publisher','Publish Date','average_rating']]

In [ ]:
book.head()

In [ ]:
book.info()

In [ ]:
book.isnull().sum()

In [ ]:
book.dropna(inplace=True)

In [ ]:
book.duplicated().sum()

In [ ]:
book.drop_duplicates(inplace = True)

In [ ]:
book.shape

In [ ]:
book['authors'] = book['authors'].apply(lambda x:x.replace(" ",""))
book['Category'] = book['Category'].apply(lambda x:x.replace(" ",""))
book['Publisher'] = book['Publisher'].apply(lambda x:x.replace(" ",""))

In [ ]:
book['Publish Date'] = pd.to_datetime(book['Publish Date']).dt.year.astype(str)

In [ ]:
book['average_rating'] = book['average_rating'].astype(str)

In [ ]:
book['Description'] = book['Description'].apply(lambda x:x.split())
book['authors'] = book['authors'].apply(lambda x:x.split())
book['Publisher'] = book['Publisher'].apply(lambda x:x.split())
book['Category'] = book['Category'].apply(lambda x:x.split())
book['Publish Date'] = book['Publish Date'].apply(lambda x:x.split())
book['average_rating'] = book['average_rating'].apply(lambda x:x.split())

In [ ]:
book['tag'] = book['Description']+book['authors']+book['Publisher']+book['Category']+book['Publish Date']+book['average_rating']

In [ ]:
book.head()

In [ ]:
book.info()
book.shape

In [ ]:
book = book[['Title','tag']]

In [ ]:
book.head()

In [ ]:
book['tag'] = book['tag'].apply(lambda x:" ".join(x))

In [ ]:
book['tag'] = book['tag'].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 18000)

In [ ]:
vector = cv.fit_transform(book['tag'])

In [ ]:
vector.shape

In [ ]:
cv.get_feature_names_out()

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
book['tag'] = book['tag'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity[0]
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
book[book['Title'] == 'This Lullaby'].index[0]

In [ ]:
def rec(b):
    book_index = book[book['Title'] == b].index[0]
    dist = similarity[book_index]
    book_list = sorted(list(enumerate(dist)),reverse=True,key=lambda x:x[1])[1:6]

    for i in book_list:
        print(book.iloc[i[0]][0])

In [ ]:
rec('This Lullaby')

In [ ]:
#https://www.kaggle.com/datasets/elvinrustam/books-dataset
#https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks